In [26]:
import pandas as pd
import configparser
import requests
import os
from datetime import date, timedelta

# AUCTION SPECIFICATIONS & RESULTS 
### Download data by API 

In [27]:
url = 'https://api.jao.eu/OWSMP/getauctions'  

config = configparser.ConfigParser()
config.read('config.ini')
token = config['DEFAULT']['AUTH_API_KEY']

headers = {
    'AUTH_API_KEY': token
}

params = {
    'horizon': 'Daily',  
    'corridor': 'PL-UA',  
    'fromdate': date.today() - timedelta(days=30), 
    'todate': date.today(),  
}

response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()
    # print(data)  
else:
    print('Error:', response.status_code, response.text)

### Creation of a date column

In [28]:
df_JAO= pd.json_normalize(data, record_path=['results'], meta=['identification'])

print(df_JAO.head(24))

   comment corridorCode productIdentification  productHour  offeredCapacity  \
0     None        PL-UA            B08-------  07:00-08:00             69.0   
1     None        PL-UA            B09-------  08:00-09:00              0.0   
2     None        PL-UA            B10-------  09:00-10:00              0.0   
3     None        PL-UA            B11-------  10:00-11:00              0.0   
4     None        PL-UA            B12-------  11:00-12:00              0.0   
5     None        PL-UA            B13-------  12:00-13:00              0.0   
6     None        PL-UA            B14-------  13:00-14:00              0.0   
7     None        PL-UA            B15-------  14:00-15:00              0.0   
8     None        PL-UA            B01-------  00:00-01:00             32.0   
9     None        PL-UA            B02-------  01:00-02:00              0.0   
10    None        PL-UA            B03-------  02:00-03:00              0.0   
11    None        PL-UA            B04-------  03:00

In [29]:
df_JAO['date'] = df_JAO['identification'].str.split('-').str[-2] 

print(df_JAO)

    comment corridorCode productIdentification  productHour  offeredCapacity  \
0      None        PL-UA            B08-------  07:00-08:00             69.0   
1      None        PL-UA            B09-------  08:00-09:00              0.0   
2      None        PL-UA            B10-------  09:00-10:00              0.0   
3      None        PL-UA            B11-------  10:00-11:00              0.0   
4      None        PL-UA            B12-------  11:00-12:00              0.0   
..      ...          ...                   ...          ...              ...   
715    None        PL-UA            B13-------  12:00-13:00              0.0   
716    None        PL-UA            B14-------  13:00-14:00              0.0   
717    None        PL-UA            B15-------  14:00-15:00              0.0   
718    None        PL-UA            B16-------  15:00-16:00             56.0   
719    None        PL-UA            B17-------  16:00-17:00            123.0   

     requestedCapacity  allocatedCapaci

In [30]:
df_JAO['date'] = '20' + df_JAO['date'] 
df_JAO['date'] = pd.to_datetime(df_JAO['date'], format='%Y%m%d')

print(df_JAO)

    comment corridorCode productIdentification  productHour  offeredCapacity  \
0      None        PL-UA            B08-------  07:00-08:00             69.0   
1      None        PL-UA            B09-------  08:00-09:00              0.0   
2      None        PL-UA            B10-------  09:00-10:00              0.0   
3      None        PL-UA            B11-------  10:00-11:00              0.0   
4      None        PL-UA            B12-------  11:00-12:00              0.0   
..      ...          ...                   ...          ...              ...   
715    None        PL-UA            B13-------  12:00-13:00              0.0   
716    None        PL-UA            B14-------  13:00-14:00              0.0   
717    None        PL-UA            B15-------  14:00-15:00              0.0   
718    None        PL-UA            B16-------  15:00-16:00             56.0   
719    None        PL-UA            B17-------  16:00-17:00            123.0   

     requestedCapacity  allocatedCapaci

In [31]:
df_JAO['startHour'] = df_JAO['productHour'].str.split('-').str[0]

print(df_JAO.head(24))

   comment corridorCode productIdentification  productHour  offeredCapacity  \
0     None        PL-UA            B08-------  07:00-08:00             69.0   
1     None        PL-UA            B09-------  08:00-09:00              0.0   
2     None        PL-UA            B10-------  09:00-10:00              0.0   
3     None        PL-UA            B11-------  10:00-11:00              0.0   
4     None        PL-UA            B12-------  11:00-12:00              0.0   
5     None        PL-UA            B13-------  12:00-13:00              0.0   
6     None        PL-UA            B14-------  13:00-14:00              0.0   
7     None        PL-UA            B15-------  14:00-15:00              0.0   
8     None        PL-UA            B01-------  00:00-01:00             32.0   
9     None        PL-UA            B02-------  01:00-02:00              0.0   
10    None        PL-UA            B03-------  02:00-03:00              0.0   
11    None        PL-UA            B04-------  03:00

In [32]:
df_JAO.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   comment                0 non-null      object        
 1   corridorCode           720 non-null    object        
 2   productIdentification  720 non-null    object        
 3   productHour            720 non-null    object        
 4   offeredCapacity        720 non-null    float64       
 5   requestedCapacity      720 non-null    float64       
 6   allocatedCapacity      720 non-null    float64       
 7   auctionPrice           720 non-null    float64       
 8   additionalMessage      0 non-null      object        
 9   identification         720 non-null    object        
 10  date                   720 non-null    datetime64[ns]
 11  startHour              720 non-null    object        
dtypes: datetime64[ns](1), float64(4), object(7)
memory usage: 67.6+ 

In [33]:
df_JAO['dateTime'] = pd.to_datetime(df_JAO['date'].astype(str) 
                                    + 'T' 
                                    + df_JAO['startHour']
                                    + ':00')

print(df_JAO)    

    comment corridorCode productIdentification  productHour  offeredCapacity  \
0      None        PL-UA            B08-------  07:00-08:00             69.0   
1      None        PL-UA            B09-------  08:00-09:00              0.0   
2      None        PL-UA            B10-------  09:00-10:00              0.0   
3      None        PL-UA            B11-------  10:00-11:00              0.0   
4      None        PL-UA            B12-------  11:00-12:00              0.0   
..      ...          ...                   ...          ...              ...   
715    None        PL-UA            B13-------  12:00-13:00              0.0   
716    None        PL-UA            B14-------  13:00-14:00              0.0   
717    None        PL-UA            B15-------  14:00-15:00              0.0   
718    None        PL-UA            B16-------  15:00-16:00             56.0   
719    None        PL-UA            B17-------  16:00-17:00            123.0   

     requestedCapacity  allocatedCapaci

In [34]:
df_JAO.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   comment                0 non-null      object        
 1   corridorCode           720 non-null    object        
 2   productIdentification  720 non-null    object        
 3   productHour            720 non-null    object        
 4   offeredCapacity        720 non-null    float64       
 5   requestedCapacity      720 non-null    float64       
 6   allocatedCapacity      720 non-null    float64       
 7   auctionPrice           720 non-null    float64       
 8   additionalMessage      0 non-null      object        
 9   identification         720 non-null    object        
 10  date                   720 non-null    datetime64[ns]
 11  startHour              720 non-null    object        
 12  dateTime               720 non-null    datetime64[ns]
dtypes: da

### Deletion of unnecessary columns

In [35]:
drop_columns = [
    'comment', 
    'corridorCode', 
    'productIdentification', 
    'productHour', 
    'additionalMessage', 
    'identification', 
    'date', 
    'startHour' 
    ]

df_cleaned_JAO = df_JAO.drop(columns=drop_columns)
print(df_cleaned_JAO)

     offeredCapacity  requestedCapacity  allocatedCapacity  auctionPrice  \
0               69.0                5.0                5.0          0.00   
1                0.0                0.0                0.0          0.00   
2                0.0                0.0                0.0          0.00   
3                0.0                0.0                0.0          0.00   
4                0.0                0.0                0.0          0.00   
..               ...                ...                ...           ...   
715              0.0                0.0                0.0          0.00   
716              0.0                0.0                0.0          0.00   
717              0.0                0.0                0.0          0.00   
718             56.0              117.0               56.0          0.07   
719            123.0              268.0              123.0          0.07   

               dateTime  
0   2024-04-09 07:00:00  
1   2024-04-09 08:00:00  
2   2024-

### Setting the "dateTime" column as the first column.

In [36]:
colums_order = [
    'dateTime',
    'offeredCapacity',
    'requestedCapacity', 
    'allocatedCapacity', 
    'auctionPrice'
    ]

df_rearraged_JAO = df_cleaned_JAO[colums_order]
print(df_rearraged_JAO)

               dateTime  offeredCapacity  requestedCapacity  \
0   2024-04-09 07:00:00             69.0                5.0   
1   2024-04-09 08:00:00              0.0                0.0   
2   2024-04-09 09:00:00              0.0                0.0   
3   2024-04-09 10:00:00              0.0                0.0   
4   2024-04-09 11:00:00              0.0                0.0   
..                  ...              ...                ...   
715 2024-05-08 12:00:00              0.0                0.0   
716 2024-05-08 13:00:00              0.0                0.0   
717 2024-05-08 14:00:00              0.0                0.0   
718 2024-05-08 15:00:00             56.0              117.0   
719 2024-05-08 16:00:00            123.0              268.0   

     allocatedCapacity  auctionPrice  
0                  5.0          0.00  
1                  0.0          0.00  
2                  0.0          0.00  
3                  0.0          0.00  
4                  0.0          0.00  
..       

### Sorting the data according to the 'dateTime' column.

In [37]:
df_sorted_JAO = df_rearraged_JAO.sort_values(by='dateTime')

print(df_sorted_JAO.iloc[42:70])

              dateTime  offeredCapacity  requestedCapacity  allocatedCapacity  \
42 2024-04-10 18:00:00              0.0                0.0                0.0   
43 2024-04-10 19:00:00              0.0                0.0                0.0   
44 2024-04-10 20:00:00              0.0                0.0                0.0   
45 2024-04-10 21:00:00              0.0                0.0                0.0   
46 2024-04-10 22:00:00            141.0              146.0              141.0   
47 2024-04-10 23:00:00             69.0                5.0                5.0   
71 2024-04-11 00:00:00             32.0                5.0                5.0   
70 2024-04-11 01:00:00              0.0                0.0                0.0   
69 2024-04-11 02:00:00              0.0                0.0                0.0   
68 2024-04-11 03:00:00             32.0                5.0                5.0   
67 2024-04-11 04:00:00             69.0                5.0                5.0   
66 2024-04-11 05:00:00      

# PSE
## Download data by url 

### Determining the time period

In [38]:
today_date = date.today()
date_one_month_ago = today_date - timedelta(days=29)

print(today_date)
print(date_one_month_ago)

today_date_str = today_date.strftime('%Y%m%d')
date_one_month_ago_str = date_one_month_ago.strftime('%Y%m%d')

print(today_date_str)
print(date_one_month_ago_str)

2024-05-08
2024-04-09
20240508
20240409


### Download data

In [39]:
# url = "https://www.pse.pl/getcsv/-/export/csv/PL_BPKD/data_od/20240410/data_do/20240425"

url = ("https://www.pse.pl/getcsv/-/export/csv/PL_BPKD/data_od/"
       +date_one_month_ago_str
       +"/data_do/"
       +today_date_str)

response = requests.get(url)
if response.status_code == 200:
    file_path = os.path.join(os.getcwd(), 'data_PSE.csv')
    with open(file_path, 'wb') as file:
        file.write(response.content)
    print("The data was saved to:", file_path)
else:
    print("Error:", response.status_code, response.reason)

The data was saved to: /home/panberet/DownloadData/data_PSE.csv


In [40]:
file_path = 'data_PSE.csv'

df_PSE = pd.read_csv(file_path, delimiter=';', encoding='windows-1250')

print(df_PSE)

           Data  Godzina  Krajowe zapotrzebowanie na moc  \
0    2024-04-09        1                           16003   
1    2024-04-09        2                           15691   
2    2024-04-09        3                           15449   
3    2024-04-09        4                           15586   
4    2024-04-09        5                           16077   
..          ...      ...                             ...   
715  2024-05-08       20                           20762   
716  2024-05-08       21                           21304   
717  2024-05-08       22                           20181   
718  2024-05-08       23                           18434   
719  2024-05-08       24                           16996   

     Suma zdolności wytwórczych jednostek wytwórczych w KSE  \
0                                                28181        
1                                                27978        
2                                                27708        
3                          

In [41]:
df_PSE.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 22 columns):
 #   Column                                                                                       Non-Null Count  Dtype 
---  ------                                                                                       --------------  ----- 
 0   Data                                                                                         720 non-null    object
 1   Godzina                                                                                      720 non-null    int64 
 2   Krajowe zapotrzebowanie na moc                                                               720 non-null    int64 
 3   Suma zdolności wytwórczych jednostek wytwórczych w KSE                                       720 non-null    int64 
 4   Suma zdolności wytwórczych JGWa                                                              720 non-null    int64 
 5   Suma zdolności wytwórczych JGFWa           

### Creation of a date using the "Data" and "Godzina" columns

In [42]:
df_PSE['dateTime'] = (pd.to_datetime(df_PSE['Data']) 
                      + pd.to_timedelta(df_PSE['Godzina'], unit='h') 
                      - pd.Timedelta(hours=1))

print(df_PSE)

           Data  Godzina  Krajowe zapotrzebowanie na moc  \
0    2024-04-09        1                           16003   
1    2024-04-09        2                           15691   
2    2024-04-09        3                           15449   
3    2024-04-09        4                           15586   
4    2024-04-09        5                           16077   
..          ...      ...                             ...   
715  2024-05-08       20                           20762   
716  2024-05-08       21                           21304   
717  2024-05-08       22                           20181   
718  2024-05-08       23                           18434   
719  2024-05-08       24                           16996   

     Suma zdolności wytwórczych jednostek wytwórczych w KSE  \
0                                                28181        
1                                                27978        
2                                                27708        
3                          

### Checking the number of hyphens and the zero value in each column.

In [43]:
count_dashes = df_PSE.eq('-').sum()
count_zeros = df_PSE.eq(0).sum()

print(count_dashes)
print("--------------------------------------------------------------------------------------------------")
print(count_zeros)

Data                                                                                             0
Godzina                                                                                          0
Krajowe zapotrzebowanie na moc                                                                   0
Suma zdolności wytwórczych jednostek wytwórczych w KSE                                           0
Suma zdolności wytwórczych JGWa                                                                  0
Suma zdolności wytwórczych JGFWa                                                               720
Suma zdolności wytwórczych JGMa                                                                  0
Suma zdolności wytwórczych JGPVa                                                               720
Sumaryczna generacja JG aktywnych: JGWa, JGFWa, JGMa i JGPVa                                     0
Sumaryczna generacja JGWa                                                                        0
Sumaryczna

### Deletion of unnecessary columns.

In [44]:
drop_columns_PSE = [
    'Data', 
    'Godzina',
    'Suma zdolności wytwórczych JGFWa',
    'Suma zdolności wytwórczych JGPVa',
    'Sumaryczna generacja JGFWa', 
    'Sumaryczna generacja JGPVa',
    'Suma mocy z wykorzystanych Ofert Redukcji Obciążenia JGOa'
    ]

df_cleaned_PSE = df_PSE.drop(columns=drop_columns_PSE)

print(df_cleaned_PSE)

     Krajowe zapotrzebowanie na moc  \
0                             16003   
1                             15691   
2                             15449   
3                             15586   
4                             16077   
..                              ...   
715                           20762   
716                           21304   
717                           20181   
718                           18434   
719                           16996   

     Suma zdolności wytwórczych jednostek wytwórczych w KSE  \
0                                                28181        
1                                                27978        
2                                                27708        
3                                                27402        
4                                                27010        
..                                                 ...        
715                                              25672        
716                        

In [45]:
df_cleaned_PSE.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 16 columns):
 #   Column                                                                                       Non-Null Count  Dtype         
---  ------                                                                                       --------------  -----         
 0   Krajowe zapotrzebowanie na moc                                                               720 non-null    int64         
 1   Suma zdolności wytwórczych jednostek wytwórczych w KSE                                       720 non-null    int64         
 2   Suma zdolności wytwórczych JGWa                                                              720 non-null    int64         
 3   Suma zdolności wytwórczych JGMa                                                              720 non-null    int64         
 4   Sumaryczna generacja JG aktywnych: JGWa, JGFWa, JGMa i JGPVa                                 720 non-null    int64  

### Change of type from "object" to "float"

In [46]:
df_cleaned_PSE['Sumaryczna moc ładowania JGMa'] = df_cleaned_PSE['Sumaryczna moc ładowania JGMa'].astype(str)
df_cleaned_PSE['Sumaryczna moc ładowania JGMa'] = df_cleaned_PSE['Sumaryczna moc ładowania JGMa'].str.replace(',', '.')
df_cleaned_PSE['Sumaryczna moc ładowania JGMa'] = df_cleaned_PSE['Sumaryczna moc ładowania JGMa'].astype(float)

print(df_cleaned_PSE)

     Krajowe zapotrzebowanie na moc  \
0                             16003   
1                             15691   
2                             15449   
3                             15586   
4                             16077   
..                              ...   
715                           20762   
716                           21304   
717                           20181   
718                           18434   
719                           16996   

     Suma zdolności wytwórczych jednostek wytwórczych w KSE  \
0                                                28181        
1                                                27978        
2                                                27708        
3                                                27402        
4                                                27010        
..                                                 ...        
715                                              25672        
716                        

In [47]:
df_cleaned_PSE.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 16 columns):
 #   Column                                                                                       Non-Null Count  Dtype         
---  ------                                                                                       --------------  -----         
 0   Krajowe zapotrzebowanie na moc                                                               720 non-null    int64         
 1   Suma zdolności wytwórczych jednostek wytwórczych w KSE                                       720 non-null    int64         
 2   Suma zdolności wytwórczych JGWa                                                              720 non-null    int64         
 3   Suma zdolności wytwórczych JGMa                                                              720 non-null    int64         
 4   Sumaryczna generacja JG aktywnych: JGWa, JGFWa, JGMa i JGPVa                                 720 non-null    int64  

### Setting the "dateTime" column as the first column.

In [48]:
rearreged_columns_PSE = (['dateTime'] 
                         + [col for col in df_cleaned_PSE if col != 'dateTime'])

df_rearreged_PSE = df_cleaned_PSE[rearreged_columns_PSE]

print(df_rearreged_PSE)

               dateTime  Krajowe zapotrzebowanie na moc  \
0   2024-04-09 00:00:00                           16003   
1   2024-04-09 01:00:00                           15691   
2   2024-04-09 02:00:00                           15449   
3   2024-04-09 03:00:00                           15586   
4   2024-04-09 04:00:00                           16077   
..                  ...                             ...   
715 2024-05-08 19:00:00                           20762   
716 2024-05-08 20:00:00                           21304   
717 2024-05-08 21:00:00                           20181   
718 2024-05-08 22:00:00                           18434   
719 2024-05-08 23:00:00                           16996   

     Suma zdolności wytwórczych jednostek wytwórczych w KSE  \
0                                                28181        
1                                                27978        
2                                                27708        
3                                      

### Table merging

In [49]:
df_merge_PSE_JAO = pd.merge(df_sorted_JAO, df_rearreged_PSE, on='dateTime')

print(df_merge_PSE_JAO.head(10))

             dateTime  offeredCapacity  requestedCapacity  allocatedCapacity  \
0 2024-04-09 00:00:00             32.0                5.0                5.0   
1 2024-04-09 01:00:00              0.0                0.0                0.0   
2 2024-04-09 02:00:00              0.0                0.0                0.0   
3 2024-04-09 03:00:00             32.0                5.0                5.0   
4 2024-04-09 04:00:00             69.0                5.0                5.0   
5 2024-04-09 05:00:00             69.0                5.0                5.0   
6 2024-04-09 06:00:00             69.0                5.0                5.0   
7 2024-04-09 07:00:00             69.0                5.0                5.0   
8 2024-04-09 08:00:00              0.0                0.0                0.0   
9 2024-04-09 09:00:00              0.0                0.0                0.0   

   auctionPrice  Krajowe zapotrzebowanie na moc  \
0           0.0                           16003   
1           0.0  

### Saving the tables to a csv file.

In [50]:
df_merge_PSE_JAO.to_csv('merge_data.csv', index=False, sep=';')